# 네이버 뉴스 제목, 링크 크롤링
#### 마지막 실행일자 : 23.11.05
#### ※ Selenium 3.x 버전에서 실행 및 테스트함

In [ ]:
import pandas as pd
import requests as req
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs #태그 형태를 읽어 올수 있는 라이브러리 불러오기

In [ ]:
## 사용자 함수
# 페이지 넘기는 함수
def setting(idx):
    try:
        url = f"url_input"# url 설정url 설정
        header = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"}
        res = req.get(url, params={'query':'신천지'}, headers = header) #request 에서 url 값 읽어오기
        soup = bs(res.text, 'html.parser') #soup 라이브러리를 html.parser로 연결하기
        driver.get(url)
        driver.find_element_by_class_name("btn_next").click()         
    except:
        pass
    
    return soup


# 네이버뉴스만 가져오기
def appendData(a_news_tit_tag, a_info_tag):  
    news_tit_tag = []
    href_tag = []
    
    for news_tit in a_news_tit_tag:
        for info in a_info_tag:        
            if info['href'].startswith("https://n.news.naver.com/") and info['class']==['info'] and info['href'] not in href_tag:
                url = info['href']
                res = req.get(url, params={'query':'input'}, headers = header) #request 에서 url 값 읽어오기
                driver.get(url)
                soup = bs(res.text, 'html.parser')
                news_tit_tag.append(soup.find("title").text)
                href_tag.append(info['href'])
                break
                
    return news_tit_tag, href_tag

In [ ]:
## 크롤링
# 크롬열기
driver = wb.Chrome()
url = "url_input" # url 설정
header = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"}
res = req.get(url, params={'query':'input'}, headers = header) #request 에서 url 값 읽어오기
driver.get(url)


time_tag_list = []
news_tit_tag_list = []
href_tag_list = []

SIZE = 200  # 크롤링할 페이지 번호 선택
# for i in range(0, SIZE, 10):
for i in range(100, SIZE):
    print('-------------------------------', i+1, '페이지 크롤링 시작')
    soup = setting(i)      
    time_tag = []
    a_info_tag = soup.select('div.info_group > a.info')
    span_info_tag = soup.findAll('span', class_='info')
    for a_info, span_info in zip(a_info_tag, span_info_tag):
        if a_info.find("i") != None or span_info.find("i") != None:
            continue
        else:
            time_tag.append(span_info.text)
            
    a_news_tit_tag = soup.findAll('a', class_="news_tit") #soup 라이브러리로 제목 읽어 오기
    news_tit_tag, href_tag = appendData(a_news_tit_tag, a_info_tag)

    for time, news_tit, href in zip(time_tag, news_tit_tag, href_tag):
        time_tag_list.append(time)
        news_tit_tag_list.append(news_tit)
        href_tag_list.append(href)
        print("일자:", time, "\n", "제목:", news_tit, "\n",  "url:", href)  
    print('-------------------------------', i+1, '페이지 크롤링 완료')   

In [ ]:
dic={"시간":time_tag_list, "제목": news_tit_tag_list,  "링크": href_tag_list}
data = pd.DataFrame(dic)
data

In [ ]:
data.to_csv("네이버_뉴스.csv", encoding="utf-8-sig")